# Compositional statistics development

This notebook is for prototyping the implementation of compositional statistics for the Compositon class.


In [ ]:
from AtomicEmbeddings.core import Embedding
from typing import Union
import pandas as pd
import numpy as np
from AtomicEmbeddings.composition import CompositionalEmbedding

%load_ext memory_profiler


CsPbI3_magpie = CompositionalEmbedding(formula="CsPbI3", embedding="magpie")

The statistics we want to implement are:

* Weighted mean: $\bar{x} = \sum_{i=1}^n w_i x_i$
* Weighted sum: $\sum_{i=1}^n w_i x_i$
* Weighted variance: $s^2 = \sum_{i=1}^n w_i (x_i - \bar{x})^2$
* Min-pooling: $\min_{i=1}^n x_i$
* Max-pooling: $\max_{i=1}^n x_i$

If we consider a ternary compound, $A_aB_bC_c$, we can represent the individual elements with a features of dimension of N, indicated by $f_{A,i}$, $f_{B,i}$ and $f_{C,i}$($i=1,...N$).

The statistics can be represented as:

* Weighted mean: $f_{mean,i} = a^{*} f_{A,i} + b^{*} f_{B,i} + c^{*} f_{C,i}$
* Weighted sum: $f_{sum,i} = a f_{A,i} + b f_{B,i} + c f_{C,i}$
* Weighted variance: $f_{var,i} = a^{*} (f_{A,i} - f_{mean,i})^2 + b^{*} (f_{B,i} - f_{mean,i})^2 + c^{*} (f_{C,i} - f_{mean,i})^2$
* Min-pooling: $f_{min,i} = \min(f_{A,i}, f_{B,i}, f_{C,i})$
* Max-pooling: $f_{max,i} = \max(f_{A,i}, f_{B,i}, f_{C,i})$

where $a^{*} = \frac{a}{a+b+c}$, $b^{*} = \frac{b}{a+b+c}$ and $c^{*} = \frac{c}{a+b+c}$, denoting the normalized stoichiometry of the compound.

## A matrix representation of the element features

We can represent the element features as a matrix, $F$, of dimension $3 \times N$, where $N$ is the number of features and 3 is the number of elements. The matrix is defined as:

$F = \begin{bmatrix} f_{A,1} & f_{A,2} & \cdots & f_{A,N} \\ f_{B,1} & f_{B,2} & \cdots & f_{B,N} \\ f_{C,1} & f_{C,2} & \cdots & f_{C,N} \end{bmatrix}$

## A matrix representation of the stoichiometry

We can represent the stoichiometry as a matrix, $S$, of dimension $1 \times 3$, where 3 is the number of elements. The matrix is defined as:

$S = \begin{bmatrix} a & b & c \end{bmatrix}$

We can represent the normalized stoichiometry as a matrix, $S^{*}$, of dimension $1 \times 3$, where 3 is the number of elements. The matrix is defined as:

$S^{*} = \begin{bmatrix} a^{*} & b^{*} & c^{*} \end{bmatrix}$

## Implementing the statistics

We can implement the statistics using `numpy`.

## A matrix representation of the the weighted mean

We can represent the weighted mean as a matrix, $F_{mean}$, of dimension $1 \times N$, where $N$ is the number of features. The matrix is defined as:

$F_{mean} = \begin{bmatrix} f_{mean,1} & f_{mean,2} & \cdots & f_{mean,N} \end{bmatrix}$

This matrix can be calculated as:

$F_{mean} = S^{*} \cdot F$

In [ ]:
# Create the matrix of element embeddings

n = int(len(CsPbI3_magpie.fractional_composition))
m = len(CsPbI3_magpie.embedding.embeddings["H"])
el_matrix = np.zeros(shape=(n, m))
for i, k in enumerate(CsPbI3_magpie.fractional_composition.keys()):
    el_matrix[i] = CsPbI3_magpie.embedding.embeddings[k]

print(f" We have {n} elements in the formula and {m} features per element.")
print(f" The shape of the element matrix is {el_matrix.shape}")
print(el_matrix)

In [ ]:
# We can calculate the weighted mean feature vector by taking the dot product of the fractional composition and the element matrix

# Get the stoichiometric vector
stoich_vector = np.array(list(CsPbI3_magpie.fractional_composition.values()))
print(f" The stoichiometric vector is {stoich_vector}")
mean_vector = np.dot(stoich_vector, el_matrix)
print(f" The mean vector is \n {mean_vector}")

In [ ]:
# We can also use numpy.average to calculate the weighted mean
mean_vector_2 = np.average(el_matrix, axis=0, weights=stoich_vector)
print(f" The mean vector is \n {mean_vector_2}")

print(mean_vector == mean_vector_2)

In [ ]:
# Time the two methods
%timeit np.dot(stoich_vector, el_matrix)
%timeit np.average(el_matrix, axis=0, weights=stoich_vector)

Using np.dot() is quicker than using np.average() to calculate the mean feature vector. As such, we will use np.dot() to calculate the weighted mean.

## A matrix representation of the the weighted sum

We can represent the weighted sum as a matrix, $F_{sum}$, of dimension $1 \times N$, where $N$ is the number of features. The matrix is defined as:

$F_{sum} = \begin{bmatrix} f_{sum,1} & f_{sum,2} & \cdots & f_{sum,N} \end{bmatrix}$

This matrix can be calculated as:

$F_{sum} = S \cdot F$


In [ ]:
# We can calculate the weighted sum feature vector by taking the dot product of the stoichiometric vector and the element matrix

stoich_vector_unweighted = np.array(list(CsPbI3_magpie.composition.values()))
print(f" The stoichiometric vector is {stoich_vector_unweighted}")

sum_vector = np.dot(stoich_vector_unweighted, el_matrix)
print(f" The sum vector is \n {sum_vector}")

## A matrix representation of the the weighted variance

We can represent the weighted variance as a matrix, $F_{var}$, of dimension $1 \times N$, where $N$ is the number of features. The matrix is defined as:

$F_{var} = \begin{bmatrix} f_{var,1} & f_{var,2} & \cdots & f_{var,N} \end{bmatrix}$

This matrix can be calculated as:

$F_{var} = S^{*} \cdot (F - F_{mean})^2$


In [ ]:
# We can calculate the weighted variance feature vector by
# 1. Subtracting the mean vector from each element embedding
# 2. Squaring the result
# 3. Taking the dot product of the squared difference and the stoichiometric vector

# 1. Subtract the mean vector from each element embedding
el_matrix_mean_subtracted = el_matrix - mean_vector

# 2. Square the result
el_matrix_mean_subtracted_squared = el_matrix_mean_subtracted**2

# 3. Take the dot product of the squared difference and the stoichiometric vector
var_vector = np.dot(stoich_vector, el_matrix_mean_subtracted_squared)
print(f" The variance vector is \n {var_vector}")

## A matrix representation of the the min-pooling

We can represent the min-pooling as a matrix, $F_{min}$, of dimension $1 \times N$, where $N$ is the number of features. The matrix is defined as:

$F_{min} = \begin{bmatrix} f_{min,1} & f_{min,2} & \cdots & f_{min,N} \end{bmatrix}$

This matrix can be calculated as:

$F_{min} = \min(F)$


In [ ]:
# We can calculate the weighted minpool feature vector by taking the minimum of each column of the element matrix

min_vector = np.min(el_matrix, axis=0)
print(f" The min vector is \n {min_vector}")

## A matrix representation of the the max-pooling

We can represent the max-pooling as a matrix, $F_{max}$, of dimension $1 \times N$, where $N$ is the number of features. The matrix is defined as:

$F_{max} = \begin{bmatrix} f_{max,1} & f_{max,2} & \cdots & f_{max,N} \end{bmatrix}$

This matrix can be calculated as:

$F_{max} = \max(F)$


In [ ]:
# We can calculate the weighted maxpool feature vector by taking the maximum of each column of the element matrix

max_vector = np.max(el_matrix, axis=0)
print(f" The max vector is \n {max_vector}")

## Other statistics

We can also implement other statistics, such as the median, mode, standard deviation, etc. However, these statistics are not as useful as the ones listed above. As such, we will not implement them.

These other statistics be represented as:

* Geometry mean: $\sqrt[N]{\prod_{i=1}^n x_i}$
* Harmonic mean: $\frac{n}{\sum_{i=1}^n \frac{1}{x_i}}$

For our ternary compounds, we can represent the geometry mean as:

* Geometry mean: $f_{gmean,i}=\sqrt[a+b+c]{f_{A,i}^{a} \cdot f_{B,i}^{b} \cdot f_{C,i}^{c}}$
* Harmonic mean: $f_{hmean,i}=\frac{a+b+c}{\frac{1}{f_{A,i}}*a + \frac{1}{f_{B,i}}*b + \frac{1}{f_{C,i}}*c}$

In [ ]:
# We can calculate the range feature vector by taking the difference between the max and min vectors

range_vector = np.max(el_matrix, axis=0) - np.min(el_matrix, axis=0)

range_vector2 = np.ptp(el_matrix, axis=0)

print(f" The range vector is \n {range_vector}")
print(f" The range vector is \n {range_vector2}")

# Time the two methods
%timeit np.max(el_matrix, axis=0) - np.min(el_matrix, axis=0)
%timeit np.ptp(el_matrix, axis=0)

Using `np.ptp()` is quicker than calculating the max and min separately. As such, we will use `np.ptp()` to calculate the range.

## Using the CompositionalEmbedding object
The feature vectors have now been integrated into the CompositionalEmbedding object.

Let's test the calculation of feature vectors

In [ ]:
def feature_vector(comp_embed, stats=["mean"]):
    """
    Computes a feature vector based on the statistics specified in the stats argument

    Args:
        stats (list): A list of strings specifying the statistics to be computed. The default is ['mean'].

    Returns:
        np.ndarray: A feature vector of the s times the dimension of the embedding vector where s is the number of statistics specified in the stats argument
    """
    if isinstance(stats, str):
        stats = [stats]
    if not isinstance(stats, list):
        raise ValueError("Stats argument must be a list of strings")
    if not all([isinstance(s, str) for s in stats]):
        raise ValueError("Stats argument must be a list of strings")
    if not all(
        [
            s
            in [
                "mean",
                "variance",
                "minpool",
                "maxpool",
                "range",
                "sum",
                "geometric_mean",
                "harmonic_mean",
            ]
            for s in stats
        ]
    ):
        raise ValueError(
            f" {[stat for stat in stats if stat not in ['mean','variance','minpool','maxpool','range','sum','geometric_mean','harmonic_mean']]} are not valid statistics."
        )
    feature_vector = []
    for s in stats:
        if s == "mean":
            feature_vector.append(comp_embed._mean_feature_vector())
        elif s == "variance":
            feature_vector.append(comp_embed._variance_feature_vector())
        elif s == "minpool":
            feature_vector.append(comp_embed._minpool_feature_vector())
        elif s == "maxpool":
            feature_vector.append(comp_embed._maxpool_feature_vector())
        elif s == "range":
            feature_vector.append(comp_embed._range_feature_vector())
        elif s == "sum":
            feature_vector.append(comp_embed._sum_feature_vector())
        elif s == "geometric_mean":
            feature_vector.append(comp_embed._geometric_mean_feature_vector())
        elif s == "harmonic_mean":
            feature_vector.append(comp_embed._harmonic_mean_feature_vector())
    return np.concatenate(feature_vector)


stats_functions_dict = {
    "mean": "_mean_feature_vector",
    "variance": "_variance_feature_vector",
    "minpool": "_minpool_feature_vector",
    "maxpool": "_maxpool_feature_vector",
    "range": "_range_feature_vector",
    "sum": "_sum_feature_vector",
    "geometric_mean": "_geometric_mean_feature_vector",
    "harmonic_mean": "_harmonic_mean_feature_vector",
}


def feature_vector_2(comp_embed, stats=["mean"]):
    """
    Computes a feature vector based on the statistics specified in the stats argument

    Args:
        stats (list): A list of strings specifying the statistics to be computed. The default is ['mean'].

    Returns:
        np.ndarray: A feature vector of the s times the dimension of the embedding vector where s is the number of statistics specified in the stats argument
    """
    if isinstance(stats, str):
        stats = [stats]
    if not isinstance(stats, list):
        raise ValueError("Stats argument must be a list of strings")
    if not all([isinstance(s, str) for s in stats]):
        raise ValueError("Stats argument must be a list of strings")
    if not all(
        [
            s
            in [
                "mean",
                "variance",
                "minpool",
                "maxpool",
                "range",
                "sum",
                "geometric_mean",
                "harmonic_mean",
            ]
            for s in stats
        ]
    ):
        raise ValueError(
            f" {[stat for stat in stats if stat not in ['mean','variance','minpool','maxpool','range','sum','geometric_mean','harmonic_mean']]} are not valid statistics."
        )
    feature_vector = []

    for s in stats:
        feature_vector.append(getattr(comp_embed, stats_functions_dict[s])())
    return np.concatenate(feature_vector)

In [ ]:
%timeit -n 100000 -r 10 feature_vector(CsPbI3_magpie, stats=["mean", "variance", "minpool", "maxpool", "range", "sum"])
%timeit -n 100000 -r 10 feature_vector_2(CsPbI3_magpie, stats=["mean", "variance", "minpool", "maxpool", "range", "sum"])

In [ ]:


%memit feature_vector(CsPbI3_magpie, stats=["mean", "variance", "minpool", "maxpool", "range", "sum"])
%memit feature_vector_2(CsPbI3_magpie, stats=["mean", "variance", "minpool", "maxpool", "range", "sum"])

Both versions of the feature vectors are execute in similar amount of time and use similar amount of memory.
I will implement, the second version of the feature vectors, due to its better readability compared to the long chain of if-elif statements in the first version.

In [ ]:
def composition_featuriser(
    data: Union[pd.DataFrame, pd.Series, CompositionalEmbedding, list],
    embedding: Union[Embedding, str] = "magpie",
    stats: Union[str, list] = ["mean"],
    inplace: bool = False,
) -> pd.DataFrame:
    """
    Computes a feature vector for a composition based on the statistics specified in the stats argument

    Args:
        data (Union[pd.DataFrame, pd.Series, list, CompositionalEmbedding]): A pandas DataFrame or Series containing a column named 'formula', a list of formula, or a CompositionalEmbedding class
        embedding (Union[Embedding, str], optional): A Embedding class or a string
        stats (Union[str, list], optional): A list of strings specifying the statistics to be computed. The default is ['mean'].
        inplace (bool, optional): Whether to perform the operation in place on the data. The default is False.

    Returns:
        Union[pd.DataFrame,list]: A pandas DataFrame containing the feature vector, unless a list of formula is passed in which case a list of feature vectors is returned
    """

    if isinstance(data, pd.DataFrame):
        if not inplace:
            data = data.copy()
        if "formula" not in data.columns:
            raise ValueError(
                "The data must contain a column named 'formula' to featurise."
            )
        data["composition"] = data["formula"].apply(
            lambda x: CompositionalEmbedding(x, embedding)
        )
        data["feature_vector"] = data["composition"].apply(
            lambda x: x.feature_vector(stats)
        )
        data.drop("composition", axis=1, inplace=True)
        return data
    elif isinstance(data, pd.Series):
        if not inplace:
            data = data.copy()
        data["composition"] = data["formula"].apply(
            lambda x: CompositionalEmbedding(x, embedding)
        )
        data["feature_vector"] = data["composition"].apply(
            lambda x: x.feature_vector(stats)
        )
        data.drop("composition", axis=1, inplace=True)
        return data
    elif isinstance(data, list):
        comps = [CompositionalEmbedding(x, embedding) for x in data]
        return [x.feature_vector(stats) for x in comps]

    elif isinstance(data, CompositionalEmbedding):
        return data.feature_vector(stats)
    else:
        raise ValueError(
            "The data must be a pandas DataFrame, Series, list or CompositionalEmbedding class."
        )

In [ ]:
formula_list = ["CsPbI3", "CsPbBr3", "CsPbCl3", "CsPbF3"]
formula_dict = {
    "formula": ["CsPbI3", "CsPbBr3", "CsPbCl3", "CsPbF3"],
    "target": [1.5, 1.6, 1.7, 1.8],
}
formula_df = pd.DataFrame(formula_dict)
formula_df

In [ ]:
isinstance(formula_list, list)

In [ ]:
f = composition_featuriser(
    formula_df, stats=["mean", "sum", "variance", "minpool", "maxpool", "range"]
)
f

In [ ]:
f_list = composition_featuriser(
    formula_list, stats=["mean", "sum", "variance", "minpool", "maxpool", "range"]
)

In [ ]:
%timeit composition_featuriser(formula_df, stats=["mean", "sum", "variance", "minpool", "maxpool", "range"])
%timeit composition_featuriser(formula_list, stats=["mean", "sum", "variance", "minpool", "maxpool", "range"])